In [58]:
import math

import numpy as np

#return the number k of the first k significant eigenvalues
#W is the sorted vector of the eigenvalues
def entropy(W):
    F = np.ones(len(W))
    for i,w in enumerate(W):
        F[i] = (W[i]**2)/(sum(x*x for x in W))
    E = -1/np.log(len(W)) * sum(f * np.log(f) for f in F)
    f_sum = 0
    for i,f in enumerate(F):
        if f_sum > E:
            return i
        f_sum += f



In [59]:
#calcualates the distance between an approximation of x made by the rows in F and x
def calculate_distance(F,x):
    return np.dot(F @ x, F@x)

In [60]:
#predicts the labels of the samples in X to the Factormatrices in F
def predict(F,X):
    labels = []
    for x in X:
        min_dist = calculate_distance(F[0], x)
        label = 0;
        for i,f in enumerate(F):
            if (calculate_distance(f,x) < min_dist):
                min_dist = calculate_distance(f,x);
                label = i;
        labels.append(label)

    return labels

In [61]:
# F - all the F matrices resulting from SVD
# x - Samplevectors
def mainFunction(F, X):
    labels = predict(F,X)
    return labels

In [62]:
#cleanedData - Matrix of cleaned Data per Desease
import matplotlib.pyplot as plt

def calcFs(cleanedData):
    f_Matrices = []
    tmp = []
    k = []
    for dataset in cleanedData:
        c, w, f = np.linalg.svd(dataset, full_matrices=False)
        k.append(entropy(w))
        tmp.append(f)
    for f in tmp:
        f_Matrices.append(f[0:int (np.floor(np.mean(k))),:])
    return f_Matrices

In [98]:
import sklearn.model_selection as ms
import sklearn.metrics
import pandas as pd
import math
#run SVD and evaluate
#either path or DataFrame has to be None (the other leads or contains the data), set normalize to True to normalize the data
#returns matthews_coef

def SVD(path="../unionMatrix_Class.csv", dataFrame=None, normalize= False):
    if path != None:
        data = pd.read_csv(path)
    else:
        data = dataFrame.copy()

    if "sample_ID" in data.columns:
        data = data.drop("sample_ID", axis=1)

    if normalize:
        labels = data["label"].copy()
        data = (data-data.mean())/data.std()
        data["label"] = labels
        for col in data.columns:
            if math.isnan(data[col][0]):
                data[col] = np.zeros(len(data[col]))
        print(data)


    print(np.mean(data.iloc[:,0]),np.std(data.iloc[:,0]))

    data = data.iloc[:,1:]
    all_data = []
    all_data.append(data[data["label"] == 0])
    all_data.append(data[data["label"] == 1])
    all_data.append(data[data["label"] == 2])
    all_data.append(data[data["label"] == 3])
    all_data.append(data[data["label"] == 4])
    train= []
    test = []
    true = []
    for dat in all_data:
        X_train, X_test, Y_train, Y_test = ms.train_test_split(dat.iloc[:,0:-1], dat.iloc[:,-1],test_size=0.2, shuffle=True)
        train.append(X_train)
        test.append(X_test)
        true = true + list(Y_test)

    F = calcFs(train)
    predicted = []
    for i in range(len(test)):
        predicted = predicted + list(mainFunction(F,test[i].values))
    return sklearn.metrics.matthews_corrcoef(true,predicted)

In [99]:
SVD()

0.003878975950349108 0.1392114697046265


-0.03138463279550994